## Evaluación del modelo entrenado en enero sobre datos de febrero

Este notebook carga el modelo entrenado previamente y evalúa su rendimiento sobre los datos de febrero de 2020, replicando el ejemplo de predicción usado en el notebook original.


In [ ]:
import pandas as pd
import joblib
from sklearn.metrics import f1_score

import sys
import os
sys.path.append(os.path.abspath(".."))

from src.features.build_features import preprocess, get_feature_names


In [12]:
# Parametros
PROJECT_ROOT = os.path.abspath("..")
target_col = "high_tip"
features = get_feature_names()
MODEL_PATH = os.path.join(PROJECT_ROOT, "models", "model_jan.pkl")


In [13]:
# Carga y preprocesamiento febrero

# Cargar datos desde el enlace original
taxi_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-02.parquet')
taxi_feb = taxi_feb.head(100_000)  # muestra de prueba

# Aplicar mismo preprocesamiento
taxi_test = preprocess(taxi_feb, target_col=target_col)


In [ ]:
loaded_model = joblib.load(MODEL_PATH)

In [15]:
# Predicciones como probabilidades
preds_proba = loaded_model.predict_proba(taxi_test[features])

# Convertir probabilidades a etiquetas (clasificación binaria)
preds_labels = [p[1] for p in preds_proba.round()]

# Calcular F1-score
score = f1_score(taxi_test[target_col], preds_labels)
print(f"F1-score en febrero: {score:.4f}")


F1-score en febrero: 0.4934


## Evaluación del modelo entrenado en enero sobre febrero

El modelo entrenado exclusivamente con datos de enero 2020 fue evaluado sobre un subconjunto de los viajes de febrero 2020, obteniendo un **F1-score de 0.4934**.

### Interpretación del resultado

Este valor indica un rendimiento **moderado**, lo que sugiere que el modelo no generaliza perfectamente a los datos de un mes distinto. Esta diferencia de desempeño puede atribuirse a varias causas:

- **Cambio en el comportamiento de los pasajeros** (por ejemplo, menor propensión a dejar propina en febrero).
- **Variación en patrones de viaje** como distancia, hora de viaje o duración, que afectan la predicción.
- **Eventos externos** como clima, restricciones, o turismo estacional.

### Conclusiones preliminares

- El modelo muestra **sensibilidad temporal**, lo que valida la necesidad de analizar el rendimiento **mes a mes**.
- Para mejorar la robustez, podría considerarse:
  - Entrenar con múltiples meses de datos.
  - Hacer validación cruzada temporal (ej. enero como train, febrero como test, marzo como valid).
  - Analizar posibles cambios en la distribución de variables entre meses.

